In [ ]:
pip install category_encoders


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 970.2 kB/s eta 0:00:00


In [ ]:
!python -m pip install -U --force pip

!pip install impyute

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [2]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from sklearn import preprocessing
from sklearn.impute import KNNImputer
#from impyute.imputation.cs import mice
from sklearn.preprocessing import OrdinalEncoder
import category_encoders as ce
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from impyute.imputation.cs import mice
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

In [3]:
data=pd.read_excel('cairouniversity_final.xlsx')


In [4]:
x = data["MRN"]  # Save the "MRN" column in x
data.drop(["MRN"], axis=1, inplace=True)  # Drop the "MRN" column from the DataFrame in place
numerical_features = []
categorical_features = []

for i in data.columns:
    if data[i].dtype in ['float64','int64']:
        numerical_features.append(i)
    else:
        categorical_features.append(i)
numerical_features = []
categorical_features = []

for i in data.columns:
    if data[i].dtype in ['float64','int64']:
        numerical_features.append(i)
    else:
        categorical_features.append(i)


In [5]:
print('Numerical features: ', numerical_features)
print('\nCategorical features: ', categorical_features)

Numerical features:  ['First_BMI', 'age', 'size-cm', 'KI67']

Categorical features:  ['DM', 'HTN', 'VTE', 'CVD', 'Others', 'family_history', 'Menopausal_state', 'Hormonal_Contraception', 'T', 'N', 'M', 'Lymphovascular_Invasion', 'Laterality', 'ER', 'PR', 'HER2', 'Unilateral_Bilateral', 'Site', 'Tumor_Type', 'Grade', 'Sentinel Lymph Node Biopsy (SLNB)']


In [6]:
data["Sentinel Lymph Node Biopsy (SLNB)"] = data["Sentinel Lymph Node Biopsy (SLNB)"].replace({'Positive': 1, 'Free': 0})
#drop response variable (Trial_2)
df_2 = data['Sentinel Lymph Node Biopsy (SLNB)']
X = data.drop('Sentinel Lymph Node Biopsy (SLNB)', axis=1)

In [7]:
#split the data (Trial_2)
numerical_features = []
categorical_features = []

for i in X.columns:
    if X[i].dtype in ['float64','int64']:
        numerical_features.append(i)
    else:
        categorical_features.append(i)


In [8]:
print('Numerical features: ', numerical_features)
print('\nCategorical features: ', categorical_features)

Numerical features:  ['First_BMI', 'age', 'size-cm', 'KI67']

Categorical features:  ['DM', 'HTN', 'VTE', 'CVD', 'Others', 'family_history', 'Menopausal_state', 'Hormonal_Contraception', 'T', 'N', 'M', 'Lymphovascular_Invasion', 'Laterality', 'ER', 'PR', 'HER2', 'Unilateral_Bilateral', 'Site', 'Tumor_Type', 'Grade']


KNN Imputation

In [9]:
# Initialize the MinMaxScaler
min_max_scaler = preprocessing.MinMaxScaler()
# Fit the scaler on the data and transform it
data[numerical_features] = min_max_scaler.fit_transform(data[numerical_features])

In [10]:
k=3 #TODO: select best k
imputer = KNNImputer(n_neighbors=k)
imputed_data = imputer.fit_transform(data[numerical_features])

In [11]:
imputed_data

array([[0.42328455, 0.48484848, 0.40287226, 0.84042553],
       [0.3697561 , 0.6969697 , 0.19375157, 0.18794326],
       [1.        , 0.45454545, 0.31216931, 0.40425532],
       ...,
       [0.38107317, 0.87878788, 0.70521542, 0.5141844 ],
       [0.20097561, 0.77272727, 0.36507937, 0.22695035],
       [0.49320325, 0.63636364, 0.27437642, 0.22340426]])

In [12]:
imputed_nm_df = pd.DataFrame(imputed_data, columns=numerical_features)

Target Encoding

In [13]:
categorical_features

['DM',
 'HTN',
 'VTE',
 'CVD',
 'Others',
 'family_history',
 'Menopausal_state',
 'Hormonal_Contraception',
 'T',
 'N',
 'M',
 'Lymphovascular_Invasion',
 'Laterality',
 'ER',
 'PR',
 'HER2',
 'Unilateral_Bilateral',
 'Site',
 'Tumor_Type',
 'Grade']

In [14]:
ordinal_features = ['T', 'N', 'M', 'Grade']
nominal_features = ['DM', 'HTN', 'VTE', 'CVD', 'Others', 'family_history', 'Menopausal_state', 'Hormonal_Contraception', 'Lymphovascular_Invasion', 'Laterality', 'ER', 'PR', 'HER2', 'Unilateral_Bilateral', 'Site', 'Tumor_Type']

# 1. Encode nominal features using TargetEncoder (or any other encoding method)
if nominal_features:
    nominal_encoder = ce.TargetEncoder(cols=nominal_features)
    encoded_nominal_features = nominal_encoder.fit_transform(X[nominal_features], df_2)
else:
    encoded_nominal_features = []

# 2. Encode ordinal features using OrdinalEncoder (if applicable)
if ordinal_features:
    ordinal_encoder = ce.OrdinalEncoder(cols=ordinal_features)
    encoded_ordinal_features = ordinal_encoder.fit_transform(X[ordinal_features])
else:
    encoded_ordinal_features = []

# 3. Apply dimensionality reduction if applicable (commented out for now)
# if len(nominal_features) + len(ordinal_features) > threshold:
#     reduced_features = dimensionality_reduction(df[nominal_features + ordinal_features])

# 4. Combine encoded features (adjust based on your needs)
encoded_u = pd.concat([encoded_nominal_features, encoded_ordinal_features], axis=1)
encoded_u.shape

(692, 20)

In [15]:
encoded_u

,DM,HTN,VTE,CVD,Others,family_history,Menopausal_state,Hormonal_Contraception,Lymphovascular_Invasion,Laterality,ER,PR,HER2,Unilateral_Bilateral,Site,Tumor_Type,T,N,M,Grade
0,0.215264,0.220339,0.215038,0.214754,0.204283,0.218750,0.222222,0.195011,0.209854,0.200573,0.237232,0.231018,0.222395,0.219136,0.206897,0.223140,1,1,1,1
1,0.215264,0.220339,0.215038,0.214754,0.183466,0.137720,0.222222,0.247012,0.209854,0.227405,0.237232,0.231018,0.222395,0.219136,0.206897,0.085291,1,1,1,2
2,0.215264,0.220339,0.215038,0.214754,0.225073,0.181819,0.215017,0.195011,0.209854,0.200573,0.047309,0.087308,0.222395,0.219136,0.142810,0.223140,2,1,1,3
3,0.193955,0.193955,0.188709,0.188709,0.204283,0.218750,0.211342,0.247012,0.209854,0.200573,0.237232,0.231018,0.222395,0.219136,0.206897,0.261261,1,1,1,4
4,0.214286,0.220339,0.215038,0.214754,0.204283,0.223350,0.222222,0.195011,0.209854,0.227405,0.237232,0.231018,0.222395,0.219136,0.206897,0.146309,3,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,0.215264,0.206767,0.215038,0.214754,0.204283,0.265846,0.222222,0.195011,0.209854,0.227405,0.237232,0.231018,0.221241,0.219136,0.142810,0.188209,4,2,1,7
688,0.215264,0.220339,0.215038,0.214754,0.204283,0.181819,0.128043,0.195011,0.209854,0.200573,0.237232,0.231018,0.222395,0.142810,0.257620,0.261261,1,1,1,4
689,0.215264,0.220339,0.215038,0.214754,0.204283,0.181819,0.128043,0.195011,0.209854,0.200573,0.237232,0.231018,0.222395,0.142810,0.206897,0.138088,3,1,1,5
690,0.215264,0.220339,0.215038,0.214754,0.204283,0.181819,0.128043,0.195011,0.231343,0.200573,0.237232,0.231018,0.222395,0.142810,0.257620,0.138088,3,2,1,4


MICE Imputation after Target Encoding




In [16]:
imputer_2 = IterativeImputer(random_state=100, max_iter=10)

In [17]:
imputed_ca = imputer_2.fit_transform(encoded_u)

In [18]:
imputed_ca

array([[0.21526419, 0.22033898, 0.21503759, ..., 1.        , 1.        ,
        1.        ],
       [0.21526419, 0.22033898, 0.21503759, ..., 1.        , 1.        ,
        2.        ],
       [0.21526419, 0.22033898, 0.21503759, ..., 1.        , 1.        ,
        3.        ],
       ...,
       [0.21526419, 0.22033898, 0.21503759, ..., 1.        , 1.        ,
        5.        ],
       [0.21526419, 0.22033898, 0.21503759, ..., 2.        , 1.        ,
        4.        ],
       [0.21526419, 0.22033898, 0.21503759, ..., 1.        , 1.        ,
        4.        ]])

In [19]:
X[categorical_features].shape

(692, 20)

In [20]:
imputed_ca_df = pd.DataFrame(imputed_ca, columns= categorical_features)

In [21]:
imputed_ca_df.shape

(692, 20)

In [22]:
imputed_ca_df

,DM,HTN,VTE,CVD,Others,family_history,Menopausal_state,Hormonal_Contraception,T,N,M,Lymphovascular_Invasion,Laterality,ER,PR,HER2,Unilateral_Bilateral,Site,Tumor_Type,Grade
0,0.215264,0.220339,0.215038,0.214754,0.204283,0.218750,0.222222,0.195011,0.209854,0.200573,0.237232,0.231018,0.222395,0.219136,0.206897,0.223140,1.0,1.0,1.0,1.0
1,0.215264,0.220339,0.215038,0.214754,0.183466,0.137720,0.222222,0.247012,0.209854,0.227405,0.237232,0.231018,0.222395,0.219136,0.206897,0.085291,1.0,1.0,1.0,2.0
2,0.215264,0.220339,0.215038,0.214754,0.225073,0.181819,0.215017,0.195011,0.209854,0.200573,0.047309,0.087308,0.222395,0.219136,0.142810,0.223140,2.0,1.0,1.0,3.0
3,0.193955,0.193955,0.188709,0.188709,0.204283,0.218750,0.211342,0.247012,0.209854,0.200573,0.237232,0.231018,0.222395,0.219136,0.206897,0.261261,1.0,1.0,1.0,4.0
4,0.214286,0.220339,0.215038,0.214754,0.204283,0.223350,0.222222,0.195011,0.209854,0.227405,0.237232,0.231018,0.222395,0.219136,0.206897,0.146309,3.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,0.215264,0.206767,0.215038,0.214754,0.204283,0.265846,0.222222,0.195011,0.209854,0.227405,0.237232,0.231018,0.221241,0.219136,0.142810,0.188209,4.0,2.0,1.0,7.0
688,0.215264,0.220339,0.215038,0.214754,0.204283,0.181819,0.128043,0.195011,0.209854,0.200573,0.237232,0.231018,0.222395,0.142810,0.257620,0.261261,1.0,1.0,1.0,4.0
689,0.215264,0.220339,0.215038,0.214754,0.204283,0.181819,0.128043,0.195011,0.209854,0.200573,0.237232,0.231018,0.222395,0.142810,0.206897,0.138088,3.0,1.0,1.0,5.0
690,0.215264,0.220339,0.215038,0.214754,0.204283,0.181819,0.128043,0.195011,0.231343,0.200573,0.237232,0.231018,0.222395,0.142810,0.257620,0.138088,3.0,2.0,1.0,4.0


In [23]:
total_imputed = pd.concat([imputed_nm_df, imputed_ca_df], axis=1)

In [24]:
encoded_data_4 = pd.concat([total_imputed,df_2], axis=1)
encoded_data_4

,First_BMI,age,size-cm,KI67,DM,HTN,VTE,CVD,Others,family_history,...,Lymphovascular_Invasion,Laterality,ER,PR,HER2,Unilateral_Bilateral,Site,Tumor_Type,Grade,Sentinel Lymph Node Biopsy (SLNB)
0,0.423285,0.484848,0.402872,0.840426,0.215264,0.220339,0.215038,0.214754,0.204283,0.218750,...,0.231018,0.222395,0.219136,0.206897,0.223140,1.0,1.0,1.0,1.0,0
1,0.369756,0.696970,0.193752,0.187943,0.215264,0.220339,0.215038,0.214754,0.183466,0.137720,...,0.231018,0.222395,0.219136,0.206897,0.085291,1.0,1.0,1.0,2.0,0
2,1.000000,0.454545,0.312169,0.404255,0.215264,0.220339,0.215038,0.214754,0.225073,0.181819,...,0.087308,0.222395,0.219136,0.142810,0.223140,2.0,1.0,1.0,3.0,0
3,0.904195,0.318182,0.289494,0.148936,0.193955,0.193955,0.188709,0.188709,0.204283,0.218750,...,0.231018,0.222395,0.219136,0.206897,0.261261,1.0,1.0,1.0,4.0,0
4,0.869463,0.439394,0.299572,0.404255,0.214286,0.220339,0.215038,0.214754,0.204283,0.223350,...,0.231018,0.222395,0.219136,0.206897,0.146309,3.0,1.0,1.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,0.369106,0.772727,0.387755,0.322695,0.215264,0.206767,0.215038,0.214754,0.204283,0.265846,...,0.231018,0.221241,0.219136,0.142810,0.188209,4.0,2.0,1.0,7.0,0
688,0.467707,0.303030,0.390275,0.042553,0.215264,0.220339,0.215038,0.214754,0.204283,0.181819,...,0.231018,0.222395,0.142810,0.257620,0.261261,1.0,1.0,1.0,4.0,0
689,0.381073,0.878788,0.705215,0.514184,0.215264,0.220339,0.215038,0.214754,0.204283,0.181819,...,0.231018,0.222395,0.142810,0.206897,0.138088,3.0,1.0,1.0,5.0,0
690,0.200976,0.772727,0.365079,0.226950,0.215264,0.220339,0.215038,0.214754,0.204283,0.181819,...,0.231018,0.222395,0.142810,0.257620,0.138088,3.0,2.0,1.0,4.0,0


In [25]:
(total_imputed.isnull().sum()).sort_values(ascending=False)

First_BMI                  0
age                        0
Tumor_Type                 0
Site                       0
Unilateral_Bilateral       0
HER2                       0
PR                         0
ER                         0
Laterality                 0
Lymphovascular_Invasion    0
M                          0
N                          0
T                          0
Hormonal_Contraception     0
Menopausal_state           0
family_history             0
Others                     0
CVD                        0
VTE                        0
HTN                        0
DM                         0
KI67                       0
size-cm                    0
Grade                      0
dtype: int64

In [26]:
#Normalization
from sklearn.preprocessing import MinMaxScaler

#Assuming 'data' is your dataset with 52 features
# Create an instance of MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler to your data and transform it
normalized_data = scaler.fit_transform(total_imputed)
scaled_features_2 = pd.DataFrame(data=normalized_data)
scaled_features_2.columns= total_imputed.columns

In [27]:
#Normalization
from sklearn.preprocessing import StandardScaler
scaled_features = StandardScaler().fit_transform(total_imputed)
scaled_features = pd.DataFrame(data=scaled_features)
scaled_features.columns= total_imputed.columns

In [28]:
print(scaled_features)

     First_BMI       age   size-cm      KI67        DM       HTN       VTE  \
0     0.708758 -0.353845  0.754617  3.903295  0.220664  0.797993  0.119240   
1     0.309435  0.796121 -0.423800 -0.599525  0.220664  0.797993  0.119240   
2     5.011053 -0.518125  0.243496  0.893258  0.220664  0.797993  0.119240   
3     4.296349 -1.257389  0.115716 -0.868715 -7.151038 -2.887513 -6.869368   
4     4.037251 -0.600266  0.172507  0.893258 -0.117833  0.797993  0.119240   
..         ...       ...       ...       ...       ...       ...       ...   
687   0.304583  1.206823  0.669430  0.330405  0.220664 -1.097869  0.119240   
688   1.040151 -1.339529  0.683628 -1.602871  0.220664  0.797993  0.119240   
689   0.393861  1.781805  2.458353  1.651885  0.220664  0.797993  0.119240   
690  -0.949667  1.206823  0.541650 -0.330335  0.220664  0.797993  0.119240   
691   1.230351  0.467559  0.030529 -0.354806  0.220664  0.797993  0.119240   

          CVD    Others  family_history  ...         M  \
0   -

Note: Alhpa is lambda

@Alhpa=1.0,random_state=42 => Accuracy=0.7769784172661871

@Alhpa=7.0,random_state=42 => Accuracy=0.7769784172661871

@Alhpa=0.5,random_state=42 => Accuracy=0.7769784172661871

In [ ]:
# x=scaled_features
# y=encoded_data_4.iloc[:, -1].astype(float).values

# # Split the data into training and test sets
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
# # Ridge_model = Ridge(alpha=1.0)
# Ridge_model = RidgeClassifier(alpha=1.0)
# Ridge_model.fit(x_train, y_train)
# # Make predictions
# y_pred = Ridge_model.predict(x_test)
# print('y_pred:\n',y_pred)
# print('y_test:\n',y_test)
# # Evaluate the model
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Accuracy: {accuracy}')


y_pred:
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
y_test:
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1.
 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0.
 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0.
 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
Accuracy: 0.7769784172661871


In [ ]:
x=scaled_features
type(x)

numpy.ndarray

In [66]:
#the cross-validation method
kf = KFold(n_splits=9) #usually from k=5-10
SCORES = []
F1_SCORES = []
x=scaled_features.astype(float).values
y=encoded_data_4.iloc[:, -1].astype(float).values
#Handling imbalance
from imblearn.over_sampling import SMOTE
sm = SMOTE()
x_new, y_new = sm.fit_resample(x, y)

# Loop over the folds
for train_index, test_index in kf.split(data):
    # Split the data into training and testing sets for this fold
    x_train, x_test = x_new[train_index], x_new[test_index]
    y_train, y_test = y_new[train_index], y_new[test_index]

    # # Train your model on the training data for this fold
    # model.fit(X_train, y_train)
    Ridge_model = RidgeClassifier(alpha=1.0,fit_intercept= False,copy_X= True,max_iter= 100,positive= True,solver= 'auto',tol= 0.095)
    Ridge_model.fit(x_train, y_train)
    # Evaluate your model on the testing data for this fold
    score = Ridge_model.score(x_test, y_test)
    # Predict the labels of the test set
    y_pred = Ridge_model.predict(x_test)

    # Calculate the F1 score for this fold
    f1 = f1_score(y_test, y_pred)

    print("Fold F1 score:", f1)
    F1_SCORES.append(f1)
    # Print the score for this fold
    print("Fold score:", score)
    SCORES.append(score)

# Calculate the average F1 score
avg_f1_scores = sum(F1_SCORES)/len(F1_SCORES)
print("Average F1 Score:", avg_f1_scores)
avg_scores = sum(SCORES)/len(SCORES)
print("Average Accuracy:",avg_scores)

Fold F1 score: 0.41791044776119407
Fold score: 0.4935064935064935
Fold F1 score: 0.5614035087719298
Fold score: 0.6753246753246753
Fold F1 score: 0.5263157894736843
Fold score: 0.6493506493506493
Fold F1 score: 0.43478260869565216
Fold score: 0.6623376623376623
Fold F1 score: 0.392156862745098
Fold score: 0.5974025974025974
Fold F1 score: 0.4
Fold score: 0.6103896103896104
Fold F1 score: 0.5599999999999999
Fold score: 0.7142857142857143
Fold F1 score: 0.5106382978723404
Fold score: 0.7012987012987013
Fold F1 score: 0.5517241379310345
Fold score: 0.6578947368421053
Average F1 Score: 0.4838812948056592
Average Accuracy: 0.6401989823042454


In [ ]:
#printing all metrics scores
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'matthews_corrcoef', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'positive_likelihood_ratio', 'neg_negative_likelihood_ratio', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weig

In [52]:
# # Define our search space
# hyperParameters = {'alpha': [1e-15, 1e-10, 1e-8, 1e-3, 1e-2, 0.1, 0.2, 0.3, 0.4, 0.5, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 20.0, 30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 100.0]}
# ourmodel = RidgeClassifier()  # Initialize a new RidgeClassifier
# ridge_grid_search = GridSearchCV(ourmodel, hyperParameters, scoring='f1', cv=9)  # cv stands for cross-validation
x=scaled_features.astype(float).values
y=encoded_data_4.iloc[:, -1].astype(float).values
# Split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
# # Fit the GridSearchCV object with your data
# ridge_grid_search.fit(x, y)
# Define your search space
# 'alpha': [1e-15, 1e-10, 1e-8, 1e-3, 1e-2, 0.1, 0.2, 0.3, 0.4, 0.5, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 20.0, 30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 100.0]
#               ,'fit_intercept':[True,False]
#               ,'copy_X':[True,False]
#               ,'max_iter': [100, 200, 300, 400, 500,600,1000,5000,10000]
# 'positive':[True,False]
param_grid = {
    'tol': [1e-2, 1e-3, 1e-4, 1e-5,0.01,0.05,0.07,0.1,1.0,1.5]  # Specify a range of values for tol
              }

# Initialize a new RidgeClassifier
ridge_model_1 = RidgeClassifier()

# Use GridSearchCV for hyperparameter tuning
ridge_grid_search = GridSearchCV(ridge_model_1, param_grid, scoring='f1', cv=9)

# Fit the model with cross-validation (use the fit method)
ridge_grid_search.fit(x_train, y_train)  # Replace X_train and y_train with your training data
print("Best Model : ", ridge_grid_search.best_estimator_)
print("Best value for lambda : ", ridge_grid_search.best_params_)
print("Best F1 Score : ", ridge_grid_search.best_score_)


Best Model :  RidgeClassifier(tol=0.01)
Best value for lambda :  {'tol': 0.01}
Best F1 Score :  0.22578832702671717
